## Notebook to test embeddings and ingestion to chromadb

In [2]:
import openai

from pandas import DataFrame,  read_parquet, read_csv, concat, ExcelWriter
import requests
##from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
#from langchain.docstore.document import Document
#from azure.data.tables import TableServiceClient, TableEntity
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
from io import BytesIO
from datetime import date
#from multiprocessing import  Pool
import ast

In [3]:
OUTLOOK_CONTENT_CONNECTION_STRING = os.environ.get('OUTLOOK_CONTENT_CONNECTION_STRING')


In [4]:
#get data from azure blob storage
def get_data(file_name):
    try:
        # Create the BlobServiceClient object which will be used
        blob_service_client = BlobServiceClient.from_connection_string(OUTLOOK_CONTENT_CONNECTION_STRING)

        container_name = 'outlookcontent'
        #get today's date
        # Create a blob client using the local file name as the name for the blob
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
        
        #download blob
        blob = blob_client.download_blob()
        #convert blob to dataframe
        df = read_parquet(BytesIO(blob.readall()))
        
        #convert blob to dataframe
        #df =read_csv(BytesIO(blob.readall()), sep=',', encoding='utf-8')
        
    except Exception as e:
        return e.message, e.args

    else:
        return df

In [ ]:
df = get_data('content_processed.parquet')
df.shape

In [ ]:
df1 = get_data('content_processed_1.parquet')
df1.shape

In [ ]:
df1.head()

In [ ]:
df.head()

In [ ]:
df["finish_reason"] = df['content_processed'].apply(lambda x: x["choices"][0]["finish_reason"])
#drop rows with finish_reason is not Error
df[df['finish_reason'] == 'Error'].shape



In [ ]:
df1["finish_reason"] = df1['content_processed'].apply(lambda x: x["choices"][0]["finish_reason"])
#drop rows with finish_reason is not Error
df1[df1['finish_reason'] == 'Error'].shape


In [ ]:
df1 = df1[df1['finish_reason'] != 'Error']
df1.shape

In [ ]:
#merge dataframes with concat
df_final = concat([df, df1], ignore_index=True)
df_final.shape

In [ ]:
#check for duplicates
df_final[df_final.duplicated(subset=['PartitionKey'])].shape


In [ ]:
#function to upload data to azure blob storage
def upload_data(df):
    #get today's date
    today = date.today().strftime('%Y-%m-%d')
    try:
        #Save to Azure Blob Storage
        # Create the BlobServiceClient object which will be used
        blob_service_client = BlobServiceClient.from_connection_string(OUTLOOK_CONTENT_CONNECTION_STRING)

        container_name = 'outlookcontent'
        
        # Create a blob client using the local file name as the name for the blob
        file_name = today + "_final_data.parquet"
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
        
        # save dataframe to csv
        #csv_file = df.to_csv(index=False)

        parquet_file = BytesIO()
        df.to_parquet(parquet_file,  engine='pyarrow')
        parquet_file.seek(0)  # change the stream position back to the beginning after writing
        response = blob_client.upload_blob(data=parquet_file, overwrite=True)

        
    except:
        df.to_parquet(today + "_outlook_data.parquet", engine='pyarrow')
    else:
        return response

In [ ]:
upload_data(df_final)

### Get completion from ChatGPT

In [ ]:
#funciton to query chatgpt with content, ask for classification and return response
def get_completion(row):
    
    prompt = f"""
                Analysiere folgende Email-Unterhaltung, getrennt durch <>, nach folgenden Kriterien:
                - Sender
                - Gesendet (Datum)
                - Betreff
                - Nachricht (nur Text, keine Signaturen, Adressen, Bilder, Links, Disclaimer oder Fussnoten)
                - Typ (Frage, Antwort, Information, Aufforderung, Werbung...)

                Antwort als JSON-Objekte in einer Liste. Liste sortiert nach Datum Gesendet, älteste zuerst. 
                Beispiel:
                [{{"Sender": "Max Mustermann", "Gesendet": "2021-01-01", "Betreff": "Test", "Nachricht": "Hallo Welt", "Typ": "Frage"}}]
                <{row['content']}>
                """
    try:
        if row['content_tt_token_lenght'] < 2000:
            model = "gpt-3.5-turbo"
            max_tokens=3800 - row['content_tt_token_lenght']
        else:
            model = "gpt-3.5-turbo-16k"
            max_tokens=15500 - row['content_tt_token_lenght']
        messages = [{"role": "user", "content": prompt}]
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0, # this is the degree of randomness of the model's output
            max_tokens=max_tokens, # this is the maximum number of tokens that the model will generate
            n=1, # this is the number of samples to return
        )
        return response
    except:
        response = {"choices": [{"finish_reason": "Error"}]}
        return response
    

In [ ]:
#drop columns finish_reason and content_processed
df.drop(columns=['finish_reason', 'content_processed'], inplace=True)
df.shape

In [ ]:
df["content_processed"] = df.apply(get_completion, axis=1)

In [ ]:
upload_data(df)

### Get embeddings

In [ ]:
df_final = df.copy()

In [ ]:
#drop rows where finish_reason is length
df_final = df_final[df_final['finish_reason'] != 'length']
df_final.dropna(subset=['finish_reason'], inplace=True)
df_final.shape

In [ ]:
#create list of dictionaries from json in content_processed column
def create_list(value):
    try:
        return ast.literal_eval(value["choices"][0]["message"]["content"])
    except:
        return []

In [ ]:
#create new column with list of dictionaries
df_final['content_processed_list'] = df_final['content_processed'].apply(create_list)


In [ ]:
#check for empty lists in content_processed_list column
df_final = df_final[df_final['content_processed_list'].apply(len) != 0]
df_final.shape

In [ ]:
#create a text from the Nachrichtkeys in the json
def create_text(value):
    output_string = ''
    for d in value:
        for key, value in d.items():
            output_string += str(key) + ': ' + str(value) + '\n'                
        output_string += '\n'  
    return output_string

In [ ]:
df_final['text'] = df_final['content_processed_list'].apply(create_text)
df_final.shape

In [ ]:
#shape of empty text
check = df_final[df_final['text'] == '']
check.reset_index(inplace=True)

In [ ]:
#convert datetime to timezone unaware
check['received_datetime'] = check['received_datetime'].dt.tz_localize(None)

In [ ]:
check.shape

In [ ]:
with ExcelWriter('empty_text.xlsx') as writer:
    check.to_excel(writer, sheet_name='empty_text')

In [ ]:
test = read_parquet('/home/bender/GIT/CAS_AML_final/db/chroma-embeddings.parquet')

In [ ]:
test.shape

In [ ]:
test

#### Split text with langchain

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, DataFrameLoader

chunk_size = 1000
chunk_overlap = 50



In [ ]:
df_final.head()

In [ ]:
df_final['display_text'] = df_final['text']

In [ ]:
df_load = df_final[['subject', 'content','conversation_id', 'web_link', 'display_text', 'text']]

In [ ]:
loader = DataFrameLoader(df_load, page_content_column="text")

In [ ]:
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
texts = text_splitter.split_documents(documents)
print(f"Split into {len(texts)} chunks of text (max. {chunk_size} tokens each)")

In [ ]:
texts

### Chromadb

In [ ]:

import os
from dotenv import load_dotenv
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import chromadb
load_dotenv()

# Define the folder for storing database
PERSIST_DIRECTORY = os.environ.get('PERSIST_DIRECTORY')

# Define the Chroma settings
CHROMA_SETTINGS = Settings(
        chroma_db_impl='duckdb+parquet',
        persist_directory=PERSIST_DIRECTORY,
        anonymized_telemetry=False
)


In [ ]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-ada-002"
            )

In [ ]:

client = chromadb.Client(CHROMA_SETTINGS)

In [ ]:
client.persist()

In [ ]:
collection = client.create_collection(name="test", embedding_function=openai_ef)
collection = client.get_collection(name="test", embedding_function=openai_ef)

In [ ]:
def add_to_collection(texts):
    for text in texts[:2]:
        collection.add(documents=text.page_content, metadatas=text.metadata, ids=text.metadata['conversation_id'])
    return True

In [ ]:
add_to_collection(texts)

In [ ]:
texts[0].metadata

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
import openai

In [ ]:
texts = texts[:10]

In [ ]:
# Create embeddings
model="text-embedding-ada-002"
#embeddings_generator = openai.Embedding.create(input = [text], model=model)
embeddings = [openai.Embedding.create(input = doc.page_content, model=model) for doc in texts]

# Extract metadata
metadata = [doc.metadata for doc in texts]

# Create DataFrame
df_embeddings = DataFrame(metadata)
df_embeddings['embedding'] = embeddings

In [ ]:
texts[0]

In [ ]:
import uuid 
#create id column
df_embeddings['uuid'] = [str(uuid.uuid4()) for _ in range(len(df_embeddings.index))]


In [ ]:
#function to upload data to azure blob storage
def upload_data(df):
    
    #Save to Azure Blob Storage
    # Create the BlobServiceClient object which will be used
    blob_service_client = BlobServiceClient.from_connection_string(OUTLOOK_CONTENT_CONNECTION_STRING)
    container_name = 'outlookcontent'
    #get today's date
    today = date.today().strftime('%Y-%m-%d')
    # Create a blob client using the local file name as the name for the blob
    file_name = today + "_outlook_ada_embeddings"
    
        
    
    extension = '.parquet'
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name+extension)
    parquet_file = BytesIO()
    df.to_parquet(parquet_file,  engine='pyarrow')
    parquet_file.seek(0)  # change the stream position back to the beginning after writing
    return blob_client.upload_blob(data=parquet_file, overwrite=True)
        

        
    #except:
    #    extension = '.csv'
    #    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name+extension)
    #    csv_file = BytesIO()
    #    df.to_csv(csv_file, index=False)
    #    csv_file.seek(0)  # change the stream position back to the beginning after writing
    #    return blob_client.upload_blob(data=csv_file, overwrite=True)
    

In [ ]:
upload_data(df_embeddings)

In [ ]:
#function to add to collection
def add_to_collection(row):
    collection.add(documents=row['content'],
    embeddings=row['embedding']['data'][0]['embedding'],
     metadatas=row[['subject', 'conversation_id', 'web_link']].to_dict(),
    ids=[str(row['uuid'])])
    return True 


In [ ]:
df_embeddings.apply(add_to_collection, axis=1)

In [ ]:
collection = client.get_collection(name="openai_ada", embedding_function=openai_ef)

In [ ]:
collection.query(
    query_texts=["Simon Galli", "Email"],
    n_results=10
)

In [ ]:
df_embeddings.columns

In [ ]:
def does_vectorstore_exist(persist_directory: str) -> bool:
    """
    Checks if vectorstore exists
    """
    if os.path.exists(os.path.join(persist_directory, 'index')):
        if os.path.exists(os.path.join(persist_directory, 'chroma-collections.parquet')) and os.path.exists(os.path.join(persist_directory, 'chroma-embeddings.parquet')):
            list_index_files = glob.glob(os.path.join(persist_directory, 'index/*.bin'))
            list_index_files += glob.glob(os.path.join(persist_directory, 'index/*.pkl'))
            # At least 3 documents are needed in a working vectorstore
            if len(list_index_files) > 3:
                return True
    return False


In [ ]:
#filter for rows where text contains "Simor Galli"

df_final[df_final['text'].str.contains("Simon Galli")]

### Check

In [5]:
df_test = get_data("")
df_test.shape

(1900, 16)

In [11]:
df_test["finish_reason"] = df_test['content_summary'].apply(lambda x: x["choices"][0]["finish_reason"])

In [12]:
df_test[df_test['finish_reason'] == 'Error'].shape

(354, 16)

In [9]:
df_test[df_test['content_summary'] == 'Error'].shape

(0, 16)

In [ ]:
df_test.head()

In [ ]:
df_test['embedding'].is_null().sum()

In [ ]:
if does_vectorstore_exist(persist_directory):
    # Update and store locally vectorstore
    print(f"Appending to existing vectorstore at {persist_directory}")
    db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS)
    collection = db.get()
    texts = process_documents([metadata['source'] for metadata in collection['metadatas']])
    print(f"Creating embeddings. May take some minutes...")
    db.add_documents(texts)
else:
    # Create and store locally vectorstore
    print("Creating new vectorstore")
    texts = process_documents()
    print(f"Creating embeddings. May take some minutes...")
    db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory, client_settings=CHROMA_SETTINGS)
db.persist()
db = None

In [ ]:
def get_openai_embedding(content, model="text-embedding-ada-002"):
    text = content
    try:
        return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
    except:
        return []



In [ ]:
df['adda_embedding'] = df['text'].apply(get_openai_embedding)

In [ ]:
df.shape

In [ ]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = array_split(df, n_cores)
    pool = Pool(n_cores)
    df = concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:

df_temp['content_ada_embedding'] = df_temp.content2embed.progress_apply(lambda x: get_openai_embedding(x, model='text-embedding-ada-002'))

df_temp.shape

In [ ]:
df_temp.head()


In [ ]:
#get rows with empty embedding
#df_temp[df_temp['content_ada_embedding'].map(len) == 0]
#get rows with PartitionKey == noreply@emeaemail.teams.microsoft.com
df_temp[df_temp['PartitionKey'] == 'noreply@emeaemail.teams.microsoft.com']

In [ ]:
df_temp.iloc[282]['web_link']

In [ ]:
#function to upload data to azure blob storage
def upload_data(df):
    try:
        #Save to Azure Blob Storage
        # Create the BlobServiceClient object which will be used
        blob_service_client = BlobServiceClient.from_connection_string(OUTLOOK_CONTENT_CONNECTION_STRING)

        container_name = 'outlookcontent'
        #get today's date
        today = date.today().strftime('%Y-%m-%d')
        # Create a blob client using the local file name as the name for the blob
        file_name = today + "_outlook_data.parquet"
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
        

        parquet_file = BytesIO()
        df.to_parquet(parquet_file,  engine='pyarrow')
        parquet_file.seek(0)  # change the stream position back to the beginning after writing
        response = blob_client.upload_blob(data=parquet_file, overwrite=True)

        
    except:
        print("error uploading data to blob storage")
    else:
        return response


In [ ]:
upload_data(df_temp)

In [ ]:
a

In [ ]:
#get data from azure blob storage
def get_data(file_name):
    try:
        # Create the BlobServiceClient object which will be used
        blob_service_client = BlobServiceClient.from_connection_string(OUTLOOK_CONTENT_CONNECTION_STRING)

        container_name = 'outlookcontent'
        #get today's date
        today = date.today().strftime('%Y-%m-%d')
        # Create a blob client using the local file name as the name for the blob
        
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
        
        #download blob
        blob = blob_client.download_blob()
        #convert blob to dataframe
        df = read_parquet(BytesIO(blob.readall()))
        
        #convert blob to dataframe
        #df =read_csv(BytesIO(blob.readall()), sep=',', encoding='utf-8')
        
    except Exception as e:
        return e.message, e.args

    else:
        return df